In [65]:
import pandas as pd
import numpy as np
import faiss
from langchain_ollama import OllamaEmbeddings
import pickle

In [74]:
df = pd.read_csv('suppliers-dataset.csv')
df.head()

,name,firstName,lastName,mobile,state,city,category,address
0,تجهیزات,فاطمه,کریمی اسبو,9122169435,تهران,اسلامشهر,لوازم آشپزخانه,چهاردانگه - قاسم آباد خشکه - بزرگراه آزادگان -...
1,لوازم آشپزخانه,منیره,رضائی,9120460544,تهران,اسلامشهر,لوازم آشپزخانه,چهاردانگه - قاسم آباد خشکه - بزرگراه آزادگان -...
2,لوازم آشپزخانه,عباسعلی,احمدی دستجردی,9123067463,تهران,تهران,لوازم آشپزخانه,تهران - چهارراه مولوی - کوچه بازارابتداءنجارها...
3,فروشگاه فیاضی,مهدی,فیاضی,9122444955,تهران,تهران,لوازم آشپزخانه,"تهران, تهران, آفتاب, صالح آباد غربی, خیابان شه..."
4,فروشگاه تک,یداله,کیهانی یاوند,9125366764,تهران,قرچک,لوازم آشپزخانه,قرچک - شهرک الغدیر - کوچه معراج دوم - خیابان...


In [75]:
embeddings_model = OllamaEmbeddings(model="bge-m3:567m")

def get_ollama_embeddings(texts):
    if not isinstance(texts, list):
        texts = [texts]
    embeddings = embeddings_model.embed_documents(texts)
    return np.array(embeddings, dtype=np.float32)

In [76]:
def clearTextData(row: pd.Series) -> str:
    return (
        'صنف: ' + str(row['category']) + ' | ' +
        'نام و نام خانوادگی: ' + str(row['firstName']) + ' ' + str(row['lastName']) + ' | ' +
        'نام مجموعه: ' + str(row['name']) + ' | ' +
        'موبایل: 0' + str(row['mobile']) + ' | ' +
        'آدرس: ' + str(row['address'])
    )

In [93]:
texts = list(df.apply(clearTextData, axis=1)) 

In [95]:
vectors = get_ollama_embeddings(texts)
dim = vectors.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(vectors)
faiss.write_index(index, "befoys_suppliers_database.bin")

In [97]:
metadata = df[['name','firstName','lastName','mobile','state','city', 'category', 'address']].to_dict(orient='records')
with open("befoys_suppliers_metadata.pkl", "wb") as f:
    pickle.dump(metadata, f)

In [ ]:
def search(query, k=5):
    query_vector = get_ollama_embeddings(query)
    distances, indices = index.search(query_vector, k)
    results = []
    for dist, idx in zip(distances[0], indices[0]):
        row = df.iloc[idx]
        results.append(clearTextData(row))
    return results